In [20]:
from grafos import Accion
from grafos import Estado
from grafos import Nodo
from grafos import Problema

Crear métodos auxiliares

In [21]:
def crear_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    return raiz

def crear_nodo_hijo(problema, padre, accion):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    padre.hijos.append(hijo)
    return hijo

def muestra_solucion(objetivo = None, problema_resolver = None):
    if not objetivo:
        print('No hay solución')
        return
    nodo = objetivo
    while nodo:
        msg = 'Estado: {0}, Costo Total: {1}'
        estado = nodo.estado.nombre
        costo_total = nodo.costo
        print(msg.format(estado, costo_total))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            msg = "<----{0} [{1}]---->"
            print(msg.format(accion, costo))
            print('-----------')
        nodo = nodo.padre

Crear la función que ejecuta el problema

In [22]:
def costo_uniforme(problema):
    raiz = crear_nodo_raiz(problema)
    frontera = [raiz,]
    explorados = set()
    while True:
        print('frontera: ', [(nodo.estado.nombre, nodo.costo) for nodo in frontera])
        if not frontera:
            return None
        nodo = frontera.pop(0)
        if problema.es_objetivo(nodo.estado):
            return nodo
        print('escoge: ', nodo.estado.nombre)
        print('------------')
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crear_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if (hijo.estado not in explorados and hijo.estado not in estados_frontera):
                frontera.append(hijo)
            else:
                buscar = [nodo for nodo in frontera if nodo.estado == hijo.estado]
                if buscar:
                    if hijo.costo < buscar[0].costo:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            frontera.sort(key = lambda nodo: nodo.costo)

# Construir estructura del problema

In [26]:
# Lo mismo que en el de amplitud, pero en lugar de una cola, usamos una pila
if __name__ == '__main__':
    # vamos a crear las acciones que puede hacer el agente
    accN = Accion('N')
    accS = Accion('S')
    accE = Accion('E')
    accO = Accion('O')
    accNE = Accion('NE')
    accSE = Accion('SE')
    accSO = Accion('SO')
    accNO = Accion('NO')

    # Vamos a crear los estados
    cedi = Estado('cedi', [accSE, accSO])
    tienda_1 = Estado('tienda_1', [accS, accSE])
    tienda_2 = Estado('tienda_2', [accO, accS, accSE])
    tienda_3 = Estado('tienda_3', [accE])
    tienda_4 = Estado('tienda_4', [accE])
    tienda_5 = Estado('tienda_5', [accNE])
    parqueadero = Estado('parqueadero', [])

    # Acciones que puede hacer
    acciones = {'cedi':{'SO':tienda_1, 'SE':tienda_2},
                'tienda_1':{'S':tienda_3, 'SE':tienda_4},
                'tienda_2':{'O':tienda_1, 'S':tienda_4, 'SE':parqueadero},
                'tienda_3':{'E':tienda_4},
                'tienda_4':{'E':tienda_5},
                'tienda_5':{'NE':parqueadero}}
    
    costos = {'cedi':{'SO':15, 'SE':50},
                'tienda_1':{'S':20, 'SE':16},
                'tienda_2':{'O':18, 'S':25, 'SE':42},
                'tienda_3':{'E':8},
                'tienda_4':{'E':12},
                'tienda_5':{'NE':30}
                }

In [27]:
objetivo = [ parqueadero]
problema = Problema(cedi, objetivo, acciones, costos)
solucion = costo_uniforme(problema)
muestra_solucion(solucion, problema)

frontera:  [('cedi', 0)]
escoge:  cedi
------------
frontera:  [('tienda_1', 15), ('tienda_2', 50)]
escoge:  tienda_1
------------
frontera:  [('tienda_4', 31), ('tienda_3', 35), ('tienda_2', 50)]
escoge:  tienda_4
------------
frontera:  [('tienda_3', 35), ('tienda_5', 43), ('tienda_2', 50)]
escoge:  tienda_3
------------
frontera:  [('tienda_5', 43), ('tienda_2', 50)]
escoge:  tienda_5
------------
frontera:  [('tienda_2', 50), ('parqueadero', 73)]
escoge:  tienda_2
------------
frontera:  [('parqueadero', 73)]
Estado: parqueadero, Costo Total: 73
<----NE [30]---->
-----------
Estado: tienda_5, Costo Total: 43
<----E [12]---->
-----------
Estado: tienda_4, Costo Total: 31
<----SE [16]---->
-----------
Estado: tienda_1, Costo Total: 15
<----SO [15]---->
-----------
Estado: cedi, Costo Total: 0
